In [2]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:70% !important; }</style>")) 

# データ読み込み等

In [56]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import os
os.chdir("C:\\Users\\takumi\\Desktop\\L-pro_repair\\L-pro\\201609-201708")

In [3]:
# db_2017 = pd.read_csv('db.csv',encoding='cp932')
# db_2018 = pd.read_csv('buhin_300_all.csv',encoding='cp932')
# pd.merge(db_2017,db_2018,how='inner')
# db_2017

In [4]:
# import os
# os.chdir("C:\\Users\\takumi\\Desktop\\L-pro_repair\\L-pro\\2017only\\2017_igai")
# pd.read_csv('df_up70_dummy_2017_500.csv')

In [57]:
# import os
# os.chdir("C:\\Users\\takumi\\Desktop\\L-pro_repair\\L-pro\\2017only\\")
df_up70 = pd.read_csv("df_up70.csv")
df_up70['受付日']=pd.to_datetime(df_up70['受付日'],format='%Y%m%d')
del df_up70['Unnamed: 0']
df_up70.index = df_up70['受付日']
df_up70.sort_index(inplace=True)
# df_flg = df_up70[(df_up70['一発修理FLG']==1)]#一発修理〇に絞る

今回は地域を受付NO登場回数が上位70%に絞っているので行数は去年よりも少なくなる

# 欠損値処理

In [58]:
#欠損値処理
df_dropna = df_up70.dropna(subset=['受付NO','部品品番','拠点','受付日','大現象名','品番グループ','修理依頼内容','一発修理FLG'])
#df_dropna['受付NO'].value_counts()
#重複行処理
df_dropna = df_dropna.drop_duplicates()

In [59]:
#受付NOが2つ以上
g2 = df_dropna[(df_dropna.duplicated('受付NO'))]
#受付NOが1つだけ
g3 = df_dropna[(~df_dropna.duplicated('受付NO'))]

#g2の中で'大現象名','品番グループ','修理依頼内容'が1パターンではないもの
#g2[~g2.duplicated(['大現象名','品番グループ','修理依頼内容'])]
#'大現象名','品番グループ','修理依頼内容'が1パターンであるもの
g4 = g2[g2.duplicated(['大現象名','品番グループ','修理依頼内容'])]
df = pd.concat([g3,g4]).sort_index()


# 学習データの絞り込み

In [60]:
df = df[('2016-09-01' <= df['受付日']) & (df['受付日'] <= '2017-08-31')]
# df = df[df['拠点'] == '横浜']
df

,受付NO,部品品番,拠点,受付日,大現象名,商品品目,品番中グループ,品番グループ,商品品番,修理依頼内容,社内メッセージ,得意先メッセージ,一発修理FLG
受付日,,,,,,,,,,,,,
2016-09-01,L2044216,A-4939,新潟,2016-09-01,漏水,DW,DT-V283ﾀｲﾌﾟ,AMG-V DT-V200ﾀｲﾌﾟ(ﾘﾄｲﾚ),DT-V283HU/BN8,ｼｬﾜ-の出が悪い。機能部より水漏れ。,【遠藤ＭＥ相談済。アポ連絡お願いします。受付Ｃ門雪乃】♯Ｂ１【９／１（木）訪問希望】【施主様...,至急訪問希望。アメージュＸリトイレ,0
2016-09-01,L2193610,E43-124,名古屋,2016-09-01,温風(ﾄﾞﾗｲ)関係,EW,CW-E53ｼﾘ-ｽﾞ,PASSO E53,CW-E53Q/BU8,1）電源が入らず、暖かくならない,伊藤ＭＥ依頼分。西ＵＣ平下,アポ連絡時に概算費用の説明をお願いします。２）電源ランプ緑色高速点滅。初回。２）の電源ランプ...,1
2016-09-01,L2193618,372-1013,三河,2016-09-01,脱臭関係,DW,DT-388Jﾀｲﾌﾟ,ｱｽﾃｵ DT-300Jﾀｲﾌﾟ(ﾘﾄｲﾚ),DT-387JH/BU8,ターボ脱臭機能が作動しなくなった。,♯Ｂ２ 西ＦＡＸ太田亜希,NaN,1
2016-09-01,L2193598,DV-J223G,三河,2016-09-01,異音関係,DW,DV-J214ﾀｲﾌﾟ,ﾍﾞｰｼｱﾊｰﾓ DV-Jﾀｲﾌﾟ,"DV-J223G-SR,GBC-J22S-SR/BW1",1階ﾄｲﾚﾀﾝｸﾚｽ洗浄水を流すと便座下奥ﾊﾞﾙﾌﾞ付近より異音。,♯Ｂ２【奥様携帯へ連絡希望。受付Ｃより奥様携帯へ受付一報するも不出。ＦＯＲＣＥにて行動入力願...,１階トイレ タンクレスＪ（Ｄ−Ｊ２２３Ｇ−ＳＲ 便器にはＧＢＣ−Ｊ２２Ｓと表記があります。）...,1
2016-09-01,L2193624,TF-10R-L,名古屋,2016-09-01,便器洗浄関係,DW,DT-V283ﾀｲﾌﾟ,AMG-V DT-V200ﾀｲﾌﾟ,DT-V251U/BU8,タンクレバーの小への動作が硬い,♯Ａ３【本日午後７時か明日午後７時にアポ連絡お願いいたします。かなり細かい方ですので、時間厳...,３０２８Ｒ２４３９と同じ部品交換の場合は１年で同じ個所が壊れているので、原因の書面（レポート...,1
2016-09-01,L2044573,TF-10A,新潟,2016-09-01,便器洗浄関係,IW,その他,その他,DT-883,洗浄ﾊﾝﾄﾞﾙが戻らなくなる,♯Ｂ２【９／２ＡＭまでに施主様へ連絡願います】流すと洗浄ハンドルが回ったまま戻らなくなる。今...,NaN,1
2016-09-01,L2044204,351-1038A,千葉,2016-09-01,電源関係,DW,DV-118Aﾀｲﾌﾟ,ｻﾃｨｽ DV-100Aﾀｲﾌﾟ,DV-115A/LR8,本体電源が入らない。,♯Ｂ２ 施主様へ本日９／１連絡希望。概算案内願います。水漏れなし。プラグ抜差ししてリモコンの...,NaN,1
2016-09-01,L2306644,50-186,大阪南,2016-09-01,便器洗浄関係,TW,その他,その他,TWC-2B,レバーをひねってもフラッパーゴムが動かない。,♯Ｂ２ 西ＦＡＸ鐙啓人,NaN,1
2016-09-01,L2306615,50-2318,大阪南,2016-09-01,漏水,DW,DV-118Aﾀｲﾌﾟ,ｻﾃｨｽ DV-100Aﾀｲﾌﾟ,DV-115A/BW1,□封水が溜まらない。,♯Ｂ２【アポ連絡は、本日又は９／２ＡＭまでに自宅へ連絡希望。自宅不在時は施主様携帯までお願い...,奥様からの入電。設置１１年。①封水が溜まらない。本体の洗浄ハンドルで流してみた所一度は改善し...,1


# dfの中で頻出部品選出

# Mecabで形態素解析

## mecab-ipadic-neologd のエントリを生成する際の正規化処理
https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp

In [61]:
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

# unicodedata.normalizeのNFKC（Normalization Form Compatibility Composition）で
# 半角カタカナ、全角英数、ローマ数字・丸数字、異体字などなどを正規化
    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]', '', s)  # remove tildes
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

In [62]:
# 大文字を小文字に変換
def lower_text(text):
    return text.lower()

# 数字を０で置換
def normalize_number(text):
    # 連続した数字を0で置換
    replaced_text = re.sub(r'\d+', '0', text)
    return replaced_text

In [63]:
df["修理依頼内容"] = df["修理依頼内容"].apply(normalize_neologd)
df["修理依頼内容"] = df["修理依頼内容"].apply(lower_text)
df["修理依頼内容"] = df["修理依頼内容"].apply(normalize_number)

# 特定文字列の置換

In [64]:
df['修理依頼内容'] = df['修理依頼内容'].str.replace('シャワ-','シャワー')
df['修理依頼内容'] = df['修理依頼内容'].str.replace('センサ-','センサー')
df['修理依頼内容'] = df['修理依頼内容'].str.replace('レバ-','レバー')

# '\dF'　1Fの階の意味と、部品等のTFを区別するため
df['修理依頼内容'] = df['修理依頼内容'].replace('\df', '階',regex=True)

# 単語の分割

In [65]:
import MeCab
def wakati_by_mecab(text):
    tagger = MeCab.Tagger('')
    tagger.parse('') 
# parseToNode()を使うと形態素の詳細情報が得らる
# parseToNode()は先頭のノード（形態素情報）を返し、surfaceで表層形、featureで形態素情報を取得
# featureは , で区切られているのでsplit()などで分割して必要な情報を抽出
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        pos = node.feature.split(",")[0]
        if pos in ["名詞", "動詞", "形容詞","副詞"]:   # 対象とする品詞
            word = node.surface
            word_list.append(word)
        node = node.next
    return word_list

In [66]:
# import MeCab
# tagger = MeCab.Tagger('') 
# node = tagger.parseToNode("あなたはあそこの渋谷ヒカリエに出かけた")
# word_list = []
# while node:
#     pos = node.feature.split(",")[0]
#     if pos in ["名詞", "動詞", "形容詞"]:   # 対象とする品詞
#         word = node.surface
#         word_list.append(word)
#     node = node.next
# word_list

In [67]:
df["修理依頼内容"] = df["修理依頼内容"].apply(wakati_by_mecab)

In [68]:
df.head()

,受付NO,部品品番,拠点,受付日,大現象名,商品品目,品番中グループ,品番グループ,商品品番,修理依頼内容,社内メッセージ,得意先メッセージ,一発修理FLG
受付日,,,,,,,,,,,,,
2016-09-01,L2044216,A-4939,新潟,2016-09-01,漏水,DW,DT-V283ﾀｲﾌﾟ,AMG-V DT-V200ﾀｲﾌﾟ(ﾘﾄｲﾚ),DT-V283HU/BN8,"[シャワー, 出が, 悪い, 機能, 部, 水, 漏れ]",【遠藤ＭＥ相談済。アポ連絡お願いします。受付Ｃ門雪乃】♯Ｂ１【９／１（木）訪問希望】【施主様...,至急訪問希望。アメージュＸリトイレ,0
2016-09-01,L2193610,E43-124,名古屋,2016-09-01,温風(ﾄﾞﾗｲ)関係,EW,CW-E53ｼﾘ-ｽﾞ,PASSO E53,CW-E53Q/BU8,"[0, 電源, 入ら, 暖かく, なら]",伊藤ＭＥ依頼分。西ＵＣ平下,アポ連絡時に概算費用の説明をお願いします。２）電源ランプ緑色高速点滅。初回。２）の電源ランプ...,1
2016-09-01,L2193618,372-1013,三河,2016-09-01,脱臭関係,DW,DT-388Jﾀｲﾌﾟ,ｱｽﾃｵ DT-300Jﾀｲﾌﾟ(ﾘﾄｲﾚ),DT-387JH/BU8,"[ターボ, 脱臭, 機能, 作動, し, なっ]",♯Ｂ２ 西ＦＡＸ太田亜希,NaN,1
2016-09-01,L2193598,DV-J223G,三河,2016-09-01,異音関係,DW,DV-J214ﾀｲﾌﾟ,ﾍﾞｰｼｱﾊｰﾓ DV-Jﾀｲﾌﾟ,"DV-J223G-SR,GBC-J22S-SR/BW1","[0階, トイレタンクレス, 洗浄, 水, 流す, 便座, 下奥, バルブ, 付近, 異音]",♯Ｂ２【奥様携帯へ連絡希望。受付Ｃより奥様携帯へ受付一報するも不出。ＦＯＲＣＥにて行動入力願...,１階トイレ タンクレスＪ（Ｄ−Ｊ２２３Ｇ−ＳＲ 便器にはＧＢＣ−Ｊ２２Ｓと表記があります。）...,1
2016-09-01,L2193624,TF-10R-L,名古屋,2016-09-01,便器洗浄関係,DW,DT-V283ﾀｲﾌﾟ,AMG-V DT-V200ﾀｲﾌﾟ,DT-V251U/BU8,"[タンク, レバー, 小, 動作, 硬い]",♯Ａ３【本日午後７時か明日午後７時にアポ連絡お願いいたします。かなり細かい方ですので、時間厳...,３０２８Ｒ２４３９と同じ部品交換の場合は１年で同じ個所が壊れているので、原因の書面（レポート...,1


### ストップワードの除去

In [69]:
import os
import urllib.request

def download_stopwords(path):
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    if os.path.exists(path):
        print('File already exists.')
    else:
        print('Downloading...')
        # Download the file from `url` and save it locally under `file_name`:
        urllib.request.urlretrieve(url, path)

def create_stopwords(file_path):     
    stop_words = [w.replace('\n','') for w in open(path, "r",encoding="utf-8_sig") if len(w) > 0]
    return stop_words    

path = "stop_words.txt"
download_stopwords(path)
stop_words = create_stopwords(path)
stop_words.remove('水')
stop_words.remove('春')
stop_words.remove('夏')
stop_words.remove('秋')
stop_words.remove('冬')
stop_words.append("0")

File already exists.


In [70]:
print(stop_words)

['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ', 'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ', 'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'ちゃん', 'てん', 'とおり', 'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば', 'なに', 'など', 'なん', 'はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ', 'へん', 'ぺん', 'ほう', 'ほか', 'まさ', 'まし', 'まとも', 'まま', 'みたい', 'みつ', 'みなさん', 'みんな', 'もと', 'もの', 'もん', 'やつ', 'よう', 'よそ', 'わけ', 'わたし', '', 'ハイ', '', '', '上', '中', '下', '字', '', '', '年', '月', '日', '時', '分', '秒', '週', '火', '木', '金', '土', '国', '都', '道', '府', '県', '市', '区', '町', '村', '', '', '各', '第', '方', '何', '的', '度', '文', '者', '性', '体', '人', '他', '今', '部', '課', '係', '外', '類', '達', '気', '室', '口', '誰', '用', '界', '会', '首', '男', '女', '別', '話', '私', '屋', '店', '家', '場', '等'

In [71]:
def remove_stopwords(words):
    words = [word for word in words if word not in stop_words]
    return words

In [72]:
df["修理依頼内容"] = df["修理依頼内容"].apply(remove_stopwords)

# ひらがな、カタカナ1文字のものを削除

In [73]:
def remove_onecharacter(a):
    remove_list = []
    for ch in a:
        if len(ch) == 1:
            name = unicodedata.name(ch)
            if "HIRAGANA" in name or "KATAKANA" in name:
                remove_list.append(ch)
    
    words = [word for word in a if word not in remove_list]
    return words

In [74]:
def remove_onecharacter(a):
    remove_list = []
    for ch in a:
        if len(ch) == 1:
            name = unicodedata.name(ch)
            if "HIRAGANA" in name or "KATAKANA" in name:
                remove_list.append(ch)
    
    words = [word for word in a if word not in remove_list]
    return words

In [75]:
df["修理依頼内容"] = df["修理依頼内容"].apply(remove_onecharacter)

In [76]:
df.to_csv("wakati_201609-201708.csv")

# 似た意味を持つ表記の統一

In [77]:
def replace_1(a):
    replace1 = [s.replace("蓋","ふた") for s in a]
    return replace1
def replace_2(a):
    replace2 = [s.replace("フタ","ふた") for s in a]
    return replace2
def replace_3(a):
    replace3 = [s.replace("尻","おしり") for s in a]
    return replace3
def replace_4(a):
    replace4 = [s.replace("漏れる","漏れ") for s in a]
    return replace4
def replace_5(a):
    replace5 = [s.replace("0階","階") for s in a]
    return replace5
def replace_6(a):
    replace6 = [s.replace("落ちる","落ち") for s in a]
    return replace6
def replace_7(a):
    replace7 = [s.replace("流す","流れ") for s in a]
    return replace7
def replace_8(a):
    replace8 = [s.replace("流せ","流れ") for s in a]
    return replace8
def replace_9(a):
    replace9 = [s.replace("流れる","流れ") for s in a]
    return replace9
def replace_10(a):
    replace10 = [s.replace("流し","流れ") for s in a]
    return replace10
def replace_11(a):
    replace11 = [s.replace("裏側","裏") for s in a]
    return replace11
def replace_12(a):
    replace12 = [s.replace("閉まっ","閉まる") for s in a]
    return replace12
def replace_13(a):
    replace13 = [s.replace("閉じる","閉まる") for s in a]
    return replace13

In [78]:
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_1)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_2)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_3)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_4)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_5)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_6)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_7)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_8)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_9)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_10)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_11)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_12)
df["修理依頼内容"] = df["修理依頼内容"].apply(replace_13)

# 頻出部品300のデータセット抽出  
# 頻出単語250を求める

In [79]:
#受付NOを除去する前のdfのコピーを取っておく
df2 = df.copy()

In [80]:
#受付NOの重複を除去(多くの部品が含まれる受付は修理依頼内容がかぶるために重複を除かなければならない)
df = df.drop_duplicates('受付NO')

どの期間の頻出部品を扱うかを決定

In [81]:
# 頻出部品３００
freq_list = df["部品品番"].value_counts()
freq_list_300 = freq_list[:300]
df300 = df[(df["部品品番"].isin(freq_list_300.index))]
# df300.to_csv("df300.csv")

buhin_freq = pd.DataFrame(freq_list)
buhin_300 = buhin_freq.head(300)
buhin_300.to_csv('buhin300_201609-201708.csv')

In [82]:
df_freq300 = df300

In [31]:
# #リスト内包表記に書き換え(計算時間を速くするため)
# y = [list(df_freq300["修理依頼内容"].values[i] for i in range(len(df_freq300)))]

In [83]:

y = []
for i in range(len(df_freq300)):
# for i in range(3):
    w = list(df_freq300["修理依頼内容"][i])
    y.append(w)

In [84]:
#     """2重のリストをフラットにする関数"""
def flatten(nested_list):
    return [e for inner_list in nested_list for e in inner_list]

y = flatten(y)

In [85]:
import collections
pd.set_option('display.max_rows', 250)
count = collections.Counter(y)
df_count = pd.DataFrame.from_dict(count, orient='index').reset_index()
df_count.columns=['単語','出現回数']
df_count2 = df_count.sort_values('出現回数',ascending=False)
df_250_new=df_count2.head(250).reset_index().drop('index',axis=1)

In [86]:
df_250 = df_250_new
df_250_new.to_csv("freq250_201609-201708.csv")
df2.to_csv("wakati_201609-201708.csv")

# 現象名・品番グループ整理

In [87]:
#大現象名・品番グループについてダミー変数化
df_up70_dummy = pd.get_dummies(df2,columns=['大現象名','品番グループ'])#ダミー変数化
df_up70_dummy.head()

,受付NO,部品品番,拠点,受付日,商品品目,品番中グループ,商品品番,修理依頼内容,社内メッセージ,得意先メッセージ,一発修理FLG,大現象名_その他の不具合,大現象名_便器洗浄関係,大現象名_便座･便蓋関係,大現象名_吐水関係,大現象名_外観関係(ｶﾞﾀﾂｷ含む),大現象名_止水不良(便鉢内漏水),大現象名_温風(ﾄﾞﾗｲ)関係,大現象名_漏水,大現象名_照明関係,大現象名_異臭関係,大現象名_異音関係,大現象名_脱臭関係,大現象名_部屋暖房関係,大現象名_電源関係,...,品番グループ_ﾀﾝｸﾚｽSX DV-113XUﾀｲﾌﾟ,品番グループ_ﾀﾝｸﾚｽSX DV-113Xﾀｲﾌﾟ,品番グループ_ﾄｲﾚｰﾅR,品番グループ_ﾌﾟﾚｱｽHSﾀｲﾌﾟ,品番グループ_ﾌﾟﾚｱｽHSﾀｲﾌﾟ(ﾘﾄｲﾚ),品番グループ_ﾌﾟﾚｱｽLSﾀｲﾌﾟ,品番グループ_ﾌﾟﾚｱｽLSﾀｲﾌﾟ(ﾘﾄｲﾚ),品番グループ_ﾍﾞｰｼｱ DT-3000ﾀｲﾌﾟ(10L),品番グループ_ﾍﾞｰｼｱ DT-3000ﾀｲﾌﾟ(8L),品番グループ_ﾍﾞｰｼｱ DT-4000ﾀｲﾌﾟ(10L),品番グループ_ﾍﾞｰｼｱ DT-B100ﾀｲﾌﾟ,品番グループ_ﾍﾞｰｼｱ DT-B200ﾀｲﾌﾟ,品番グループ_ﾍﾞｰｼｱ DT-BA183ﾀｲﾌﾟ,品番グループ_ﾍﾞｰｼｱVX DT-B100ﾀｲﾌﾟ,品番グループ_ﾍﾞｰｼｱｸﾘｴ DT-B100ﾀｲﾌﾟ,品番グループ_ﾍﾞｰｼｱﾊｰﾓ DV-D113ﾀｲﾌﾟ,品番グループ_ﾍﾞｰｼｱﾊｰﾓ DV-D114ﾀｲﾌﾟ,品番グループ_ﾍﾞｰｼｱﾊｰﾓ DV-DMﾀｲﾌﾟ(ﾏﾝｼｮﾝ用),品番グループ_ﾍﾞｰｼｱﾊｰﾓ DV-JMﾀｲﾌﾟ(ﾏﾝｼｮﾝ用),品番グループ_ﾍﾞｰｼｱﾊｰﾓ DV-Jﾀｲﾌﾟ,品番グループ_ﾘﾄｲﾚ DT-4000ﾀｲﾌﾟ(ﾘﾄｲﾚ),品番グループ_ﾘﾌﾚｯｼｭｻﾃｨｽ,品番グループ_ﾘﾌﾚｯｼｭｻﾃｨｽ2011,品番グループ_ﾘﾌﾚｯｼｭｻﾃｨｽ2016,品番グループ_ﾚｼﾞｵ DV-R100ﾀｲﾌﾟ
受付日,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-09-01,L2044216,A-4939,新潟,2016-09-01,DW,DT-V283ﾀｲﾌﾟ,DT-V283HU/BN8,"[シャワー, 出が, 悪い, 機能, 水, 漏れ]",【遠藤ＭＥ相談済。アポ連絡お願いします。受付Ｃ門雪乃】♯Ｂ１【９／１（木）訪問希望】【施主様...,至急訪問希望。アメージュＸリトイレ,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-09-01,L2193610,E43-124,名古屋,2016-09-01,EW,CW-E53ｼﾘ-ｽﾞ,CW-E53Q/BU8,"[電源, 入ら, 暖かく, なら]",伊藤ＭＥ依頼分。西ＵＣ平下,アポ連絡時に概算費用の説明をお願いします。２）電源ランプ緑色高速点滅。初回。２）の電源ランプ...,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-09-01,L2193618,372-1013,三河,2016-09-01,DW,DT-388Jﾀｲﾌﾟ,DT-387JH/BU8,"[ターボ, 脱臭, 機能, 作動, なっ]",♯Ｂ２ 西ＦＡＸ太田亜希,NaN,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-09-01,L2193598,DV-J223G,三河,2016-09-01,DW,DV-J214ﾀｲﾌﾟ,"DV-J223G-SR,GBC-J22S-SR/BW1","[階, トイレタンクレス, 洗浄, 水, 流れ, 便座, 下奥, バルブ, 付近, 異音]",♯Ｂ２【奥様携帯へ連絡希望。受付Ｃより奥様携帯へ受付一報するも不出。ＦＯＲＣＥにて行動入力願...,１階トイレ タンクレスＪ（Ｄ−Ｊ２２３Ｇ−ＳＲ 便器にはＧＢＣ−Ｊ２２Ｓと表記があります。）...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2016-09-01,L2193624,TF-10R-L,名古屋,2016-09-01,DW,DT-V283ﾀｲﾌﾟ,DT-V251U/BU8,"[タンク, レバー, 小, 動作, 硬い]",♯Ａ３【本日午後７時か明日午後７時にアポ連絡お願いいたします。かなり細かい方ですので、時間厳...,３０２８Ｒ２４３９と同じ部品交換の場合は１年で同じ個所が壊れているので、原因の書面（レポート...,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 単語の特徴ベクトル作成

In [89]:
#頻出単語ファイル読み込み(途中から始めるときはここでデータを読み込む)
df_250 = pd.read_csv('freq250_201609-201708.csv').drop("Unnamed: 0",axis=1)
df_250

,単語,出現回数
0,水,56053
1,漏れ,37996
2,便器,23276
3,タンク,21061
4,階,19021
5,ノズル,12466
6,床,11866
7,流れ,11566
8,出,10925
9,便座,10557


In [90]:
#単語の列を作る
for i in range(250):
    df_up70_dummy[df_250.iloc[i]["単語"]] = 0

In [93]:
#この後の作業でかなり時間がかかるので念のためcsvに書き出し
df_up70_dummy.to_csv('df_up70_dummy_201609-201708.csv')
test = df_up70_dummy.copy()

In [92]:
test.head()

,受付NO,部品品番,拠点,受付日,商品品目,品番中グループ,商品品番,修理依頼内容,社内メッセージ,得意先メッセージ,一発修理FLG,大現象名_その他の不具合,大現象名_便器洗浄関係,大現象名_便座･便蓋関係,大現象名_吐水関係,大現象名_外観関係(ｶﾞﾀﾂｷ含む),大現象名_止水不良(便鉢内漏水),大現象名_温風(ﾄﾞﾗｲ)関係,大現象名_漏水,大現象名_照明関係,大現象名_異臭関係,大現象名_異音関係,大現象名_脱臭関係,大現象名_部屋暖房関係,大現象名_電源関係,...,壁,強,濡れる,開く,出て,よく,フルオート,劣化,量,水音,繋ぎ,つく,周り,スイッチ,上げ,吸い込み,弱く,軸,グラグラ,パイプ,バルブ,調整,掃除,脱臭,左下
受付日,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-09-01,L2044216,A-4939,新潟,2016-09-01,DW,DT-V283ﾀｲﾌﾟ,DT-V283HU/BN8,"[シャワー, 出が, 悪い, 機能, 水, 漏れ]",【遠藤ＭＥ相談済。アポ連絡お願いします。受付Ｃ門雪乃】♯Ｂ１【９／１（木）訪問希望】【施主様...,至急訪問希望。アメージュＸリトイレ,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-09-01,L2193610,E43-124,名古屋,2016-09-01,EW,CW-E53ｼﾘ-ｽﾞ,CW-E53Q/BU8,"[電源, 入ら, 暖かく, なら]",伊藤ＭＥ依頼分。西ＵＣ平下,アポ連絡時に概算費用の説明をお願いします。２）電源ランプ緑色高速点滅。初回。２）の電源ランプ...,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-09-01,L2193618,372-1013,三河,2016-09-01,DW,DT-388Jﾀｲﾌﾟ,DT-387JH/BU8,"[ターボ, 脱臭, 機能, 作動, なっ]",♯Ｂ２ 西ＦＡＸ太田亜希,NaN,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-09-01,L2193598,DV-J223G,三河,2016-09-01,DW,DV-J214ﾀｲﾌﾟ,"DV-J223G-SR,GBC-J22S-SR/BW1","[階, トイレタンクレス, 洗浄, 水, 流れ, 便座, 下奥, バルブ, 付近, 異音]",♯Ｂ２【奥様携帯へ連絡希望。受付Ｃより奥様携帯へ受付一報するも不出。ＦＯＲＣＥにて行動入力願...,１階トイレ タンクレスＪ（Ｄ−Ｊ２２３Ｇ−ＳＲ 便器にはＧＢＣ−Ｊ２２Ｓと表記があります。）...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-09-01,L2193624,TF-10R-L,名古屋,2016-09-01,DW,DT-V283ﾀｲﾌﾟ,DT-V251U/BU8,"[タンク, レバー, 小, 動作, 硬い]",♯Ａ３【本日午後７時か明日午後７時にアポ連絡お願いいたします。かなり細かい方ですので、時間厳...,３０２８Ｒ２４３９と同じ部品交換の場合は１年で同じ個所が壊れているので、原因の書面（レポート...,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# ここからAnacondaプロンプト

In [41]:
#anaconda 
#頻出単語ファイル読み込み(途中から始めるときはここでデータを読み込む)
df_250 = pd.read_csv('freq250_201609-201708.csv').drop("Unnamed: 0",axis=1)
df_up70_dummy =pd.read_csv("df_up70_dummy_201609-201708.csv")
test = df_up70_dummy.copy()

修理依頼内容に頻出単語が含まれていたら1を当てる  

In [37]:
'''
for i in tqdm(range(len(df_up70_dummy))):
# for i in tqdm(range(10)):
#     print(df_up70_dummy.iloc[i]["修理依頼内容"])
    a = test.iloc[i]["修理依頼内容"]
    for k in a:
        for p in range(len(df_250)):
            if k == df_250.iloc[p]["単語"]:
                test[df_250.iloc[p]["単語"]][i] = 1
'''

100%|████████████████████████████████████████████████████████████████████| 173965/173965 [75:15:26<00:00,  1.56s/it]


# 高速化

for文の中のlocやilocはかなり時間がかかるのでnumpyをベースにしたコードに書き換え

In [ ]:
test_syuri = test["修理依頼内容"].values 
#test_syuri = test["出現回数"].values   #anaconda用に変えた
df_250_tango = df_250["単語"].values  
#df_250_tango = df_250["出現回数"].values      #
for i in tqdm(range(len(df_up70_dummy))):
    a = test_syuri[i]
    for k in a:
        for p in range(len(df_250)):
            if k == df_250_tango[p]:
                test[df_250_tango[p]][i] = 1  
                
# for i in tqdm(range(10)):
#     print(df_up70_dummy.iloc[i]["修理依頼内容"])

In [115]:
#test.iloc[1]["修理依頼内容"]
#test["修理依頼内容"].values[1]
#df_freq300["修理依頼内容"].values[i]
df_250.iloc[1]["単語"]
df_250["単語"].values[1]

'漏れ'

In [38]:
test.to_csv("201609-201708ex_data.csv")

In [42]:
#exdata = pd.read_csv("ex_data.csv",encoding='cp932',index_col=[0])
exdata = pd.read_csv("201609-201708ex_data.csv",index_col=[0])

In [43]:
exdata.head()

,受付日,受付NO,部品品番,拠点,受付日.1,商品品目,品番中グループ,商品品番,修理依頼内容,社内メッセージ,...,軸,悪く,左下,劣化,掃除,開い,暖房,ガタガタ,戻り,ストレ
0,2017-03-01,L2232644,331-1026,岐阜,2017-03-01,DW,DT-Mﾀｲﾌﾟ,DT-M182PM/BB7,"['水', '漏れ']",♯Ｂ２ 【至急、連絡お願いします】完了報告を必ずして下さい。添付：地図。受付２Ｔ三浦千春,...,0,0,0,0,0,0,0,0,0,0
1,2017-03-01,L2232654,E63-126B,三重,2017-03-01,EW,CW-E63ｼﾘｰｽﾞ,CW-E61/BN8,"['便器', '床', '設置', 'ロー', 'タンク', '流れ', 'にじみで', '...",♯Ｂ２ 請求に関して、使用上による場合は施主様へ直収、施工不良の場合はＹＳＫ建築事務所様へ請...,...,0,0,0,0,0,0,0,0,0,0
2,2017-03-01,L2232657,343-1008D,名古屋,2017-03-01,DW,DV-118Aﾀｲﾌﾟ,DV-115A/BU8,"['便器', '水', '漏れ']",♯Ｂ２【至急対応希望】サティス。便器部から水漏れ。添付：画像。受付２Ｔ三柳沙織,...,0,0,0,0,0,0,0,0,0,0
3,2017-03-01,L2232629,331-1022A,名古屋,2017-03-01,DW,DT-388Gﾀｲﾌﾟ,DT-385GU/BU8,"['便器', '床', '設置', '水', '出', 'くる']",＃Ｂ２【本日もしくは明日のお昼までに、ご自宅へご連絡お願いします。】便器と床の設置面から時々...,...,0,0,0,0,0,0,0,0,0,0
4,2017-03-01,L2232637,34-1234,三河,2017-03-01,AW,その他,DT-810UZ,"['フロ', '管', 'ぐらぐら', '水', '漏れ']",♯Ｂ２ 受付２Ｔ佐藤由佳子,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#dataset=dataset.drop(["拠点","受付日.1","品番中グループ","商品品番","商品品目","修理依頼内容","社内メッセージ","得意先メッセージ","一発修理FLG"],axis=1)
#dataset = dataset.drop(["いる","する","なる","てる"],axis=1)

In [44]:
dataset=test.drop(["拠点","受付日.1","品番中グループ","商品品番","商品品目","修理依頼内容","社内メッセージ","得意先メッセージ","一発修理FLG"],axis=1)
dataset = dataset.drop(["いる","する","なる","てる"],axis=1)

In [40]:
dataset.to_csv("201609_201708_dataset.csv")

NameError: name 'dataset' is not defined

この作業はいらないかもしれないですが、頻出かどうかは説明するときに必要になると思うので、書いておきます。

In [12]:
# #頻出部品データの読み込み,頻出部品かどうかについてのラベル振り分け
# df_300 = pd.read_csv("df_300.csv").drop("部品品番",axis=1)
# df_train2["label"] = df_train2["部品品番"].where(df_train2["部品品番"].isin(df_300["Unnamed: 0"]) ,0).where(~df_train2["部品品番"].isin(df_300["Unnamed: 0"]) ,1)

In [94]:
#df = pd.read_csv("201509-201609dataset.csv",encoding='cp932',index_col=[0])
df = pd.read_csv("201609_201708_dataset.csv",index_col=[0])

### ここまで

### ここから

# テストデータ作成

テストデータにはない特徴量がテストデータには含まれているので
その分を除いて学習させなければならない

In [95]:
#テストデータ読み込み
df_test = pd.read_csv('lixil_test.csv',encoding='cp932').dropna(subset=['受付NO','部品品番','拠点','受付日','大現象名','品番グループ','修理依頼内容','一発修理FLG'])
#重複行処理
df_test = df_test.drop_duplicates()
df_test

,受付NO,部品品番,拠点,受付日,大現象名,商品品目,品番中グループ,品番グループ,商品品番,修理依頼内容,社内メッセージ,得意先メッセージ,一発修理FLG
0,L2410076,H20-126,横浜,20170905,漏水,EW,CW-H22ｼﾘ-ｽﾞ,CW H22,CW-H22/BW1,おまかせ点検,【９／５再：施主様（女性）より早めアポ希望と入電。ＭＥメ済。受付Ｃ田井中】♯Ｂ２【本日（９／...,プラグ抜き差し復旧せず。アポ時概算費用案内希望。受付Ｃ伊藤留【ｉ－ｍａｔｅ獲得案内不可】【９...,1
1,L2410101,50-2318,横浜,20170902,漏水,DW,DV-D113ﾀｲﾌﾟ,ﾍﾞｰｼｱﾊｰﾓ DV-D113ﾀｲﾌﾟ,DV-D113G/BW1,左の上の方から床に水漏れ※流した時のみ,【９／２再：施主様（女性）よりアポ催促。取付３年位なので無償対応希望。現場判断になる旨案内、...,クラシアン様から情報伝達があり、「左の上の方」がどの部分を指すのか不明。止水栓案内済。依／ツ...,1
2,L2410131,355-1054A,横浜,20170912,ｼｬﾜｰ･ﾋﾞﾃﾞ関係,DW,DT-B183ﾀｲﾌﾟ,ﾍﾞｰｼｱVX DT-B100ﾀｲﾌﾟ,DT-B183XU/BU8,ﾘﾓｺﾝが作動しない,【唐澤ＭＥ相談済分、奥様本日１４時以降連絡後の訪問了承済みです。作動する時としない時があるが...,【９／１１：ＴＥＣ清水様受付分】,1
3,L2410173,343-1001D-PSET,横浜,20170926,漏水,DW,DT-3864ﾀｲﾌﾟ,ﾍﾞｰｼｱ DT-3000ﾀｲﾌﾟ(10L),DT-3873/SN7,ﾀﾝｸから水漏れ,♯Ｂ２【本日（９／２６）ＡＭ中に施主様へアポ連絡願います】製造ＮＯ：Ｅ７３２６９５５。受付Ｃ伊藤留,NaN,1
4,L2410209,TF-10R-L,横浜,20170921,止水不良(便鉢内漏水),AW,その他,その他,DT-883,□手洗い吐水口より水が出ない□ﾀﾝｸ内で水の音,♯Ｂ２【９／２１（木）１５：３０以降に奥様携帯へアポ連絡願います】③浮玉の線が切れている。受...,【９／２０：ＴＥＣ浜田様受付分】,1
5,L2410209,TF-20B,横浜,20170921,止水不良(便鉢内漏水),AW,その他,その他,DT-883,□手洗い吐水口より水が出ない□ﾀﾝｸ内で水の音,♯Ｂ２【９／２１（木）１５：３０以降に奥様携帯へアポ連絡願います】③浮玉の線が切れている。受...,【９／２０：ＴＥＣ浜田様受付分】,1
6,L2410214,313-1022,横浜,20170919,漏水,DW,DV-S428ﾀｲﾌﾟ,ｻﾃｨｽ DV-S400ﾀｲﾌﾟ,DV-S414/BW1,漏水（洗浄時のみ）,♯Ｂ２【本日（９／１９）施主様携帯へアポ連絡願います】後方あたりより漏水との事。ストレーナー...,【９／１８：ＴＥＣ生賀様受付分】,1
7,L2410214,A-6858,横浜,20170919,漏水,DW,DV-S428ﾀｲﾌﾟ,ｻﾃｨｽ DV-S400ﾀｲﾌﾟ,DV-S414/BW1,漏水（洗浄時のみ）,♯Ｂ２【本日（９／１９）施主様携帯へアポ連絡願います】後方あたりより漏水との事。ストレーナー...,【９／１８：ＴＥＣ生賀様受付分】,1
8,L2410278,50-1001-2,横浜,20170928,止水不良(便鉢内漏水),DW,DT-4864ﾀｲﾌﾟ,ﾍﾞｰｼｱ DT-4000ﾀｲﾌﾟ(10L),DT-4573/SG6+GBC-270S,止水不良/ﾛ-タンク内に水が流れて音がする。便器内へﾁｮﾛﾁｮﾛ,♯Ｂ２．９／２８本日奥様携帯へ連絡お願いします。着信あれば折電しますとの事。２０１４年に同商...,NaN,1
9,L2410278,A-7664,横浜,20170928,止水不良(便鉢内漏水),DW,DT-4864ﾀｲﾌﾟ,ﾍﾞｰｼｱ DT-4000ﾀｲﾌﾟ(10L),DT-4573/SG6+GBC-270S,止水不良/ﾛ-タンク内に水が流れて音がする。便器内へﾁｮﾛﾁｮﾛ,♯Ｂ２．９／２８本日奥様携帯へ連絡お願いします。着信あれば折電しますとの事。２０１４年に同商...,NaN,1


In [96]:
#受付NOが2つ以上
t2 = df_test[(df_test.duplicated('受付NO'))]
#受付NOが1つだけ
t3 = df_test[(~df_test.duplicated('受付NO'))]

#g2の中で'大現象名','品番グループ','修理依頼内容'が1パターンではないもの
#g2[~g2.duplicated(['大現象名','品番グループ','修理依頼内容'])]
#'大現象名','品番グループ','修理依頼内容'が1パターンであるもの
t4 = t2[t2.duplicated(['大現象名','品番グループ','修理依頼内容'])]
df_test = pd.concat([t3,t4]).sort_index()

In [98]:
#頻出単語・頻出部品読み込み
#df_250 = pd.read_csv('freq250_201509_201609.csv',encoding='cp932').drop("Unnamed: 0",axis=1)
df_250 = pd.read_csv('freq250_201609-201708.csv').drop("Unnamed: 0",axis=1)
#buhin300 = pd.read_csv("buhin300_201509_201609.csv",encoding='cp932').drop("部品品番",axis=1)
buhin300 = pd.read_csv("buhin300_201609-201708.csv").drop("部品品番",axis=1)

In [99]:
#df['品番グループ']
df.columns[(df.columns.str.contains('品番グループ'))]#.isin(df_test['品番グループ'])

Index(['品番グループ_(G)(H)BC-270S', '品番グループ_(G)(H)BC-320S', '品番グループ_(G)(H)BC-340SU',
       '品番グループ_(G)(H)BC-370SX', '品番グループ_(G)(H)BC-900S', '品番グループ_(G)(H)BC-901S',
       '品番グループ_(G)(H)BC-941', '品番グループ_(G)(H)BC-A10H', '品番グループ_(G)(H)BC-A10S',
       '品番グループ_(G)(H)BC-E11H',
       ...
       '品番グループ_ﾍﾞｰｼｱﾊｰﾓ DV-D113ﾀｲﾌﾟ', '品番グループ_ﾍﾞｰｼｱﾊｰﾓ DV-D114ﾀｲﾌﾟ',
       '品番グループ_ﾍﾞｰｼｱﾊｰﾓ DV-DMﾀｲﾌﾟ(ﾏﾝｼｮﾝ用)',
       '品番グループ_ﾍﾞｰｼｱﾊｰﾓ DV-JMﾀｲﾌﾟ(ﾏﾝｼｮﾝ用)', '品番グループ_ﾍﾞｰｼｱﾊｰﾓ DV-Jﾀｲﾌﾟ',
       '品番グループ_ﾘﾄｲﾚ DT-4000ﾀｲﾌﾟ(ﾘﾄｲﾚ)', '品番グループ_ﾘﾌﾚｯｼｭｻﾃｨｽ',
       '品番グループ_ﾘﾌﾚｯｼｭｻﾃｨｽ2011', '品番グループ_ﾘﾌﾚｯｼｭｻﾃｨｽ2016',
       '品番グループ_ﾚｼﾞｵ DV-R100ﾀｲﾌﾟ'],
      dtype='object', length=368)

In [100]:
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(normalize_neologd)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(lower_text)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(normalize_number)
df_test['修理依頼内容'] = df_test['修理依頼内容'].str.replace('シャワ-','シャワー')
df_test['修理依頼内容'] = df_test['修理依頼内容'].str.replace('センサ-','センサー')
df_test['修理依頼内容'] = df_test['修理依頼内容'].str.replace('レバ-','レバー')
# '\dF'　1Fの階の意味と、部品等のTFを区別するため
df_test['修理依頼内容'] = df_test['修理依頼内容'].replace('\df', '階',regex=True)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(wakati_by_mecab)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(remove_onecharacter)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_1)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_2)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_3)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_4)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_5)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_6)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_7)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_8)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_9)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_10)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_11)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_12)
df_test["修理依頼内容"] = df_test["修理依頼内容"].apply(replace_13)

In [101]:
df_test_cl = df_test.copy()
# df_test_cl['部品品番'] = df_test_cl['部品品番'].str.replace(',','')

#大現象名・品番グループについてダミー変数化
df_test_dummy = pd.get_dummies(df_test_cl,columns=['大現象名','品番グループ'])#ダミー変数化

pd.set_option('display.max_columns', 50)

In [102]:
for i in range(250):
    df_test_dummy[df_250.iloc[i]["単語"]] = 0    

# 高速化

In [103]:
df_test_dummy_syuri = df_test_dummy["修理依頼内容"].values
df_250_tango = df_250["単語"].values
for i in tqdm(range(len(df_test_dummy))):
# for i in tqdm(range(10)):
#     print(df_up70_dummy.iloc[i]["修理依頼内容"])
    a = df_test_dummy_syuri[i]
    for k in a:
        for p in range(len(df_250)):
            if k == df_250_tango[p]:
                df_test_dummy[df_250_tango[p]][i] = 1 

100%|██████████████████████████████████████████████████████████████████████████████| 1191/1191 [00:24<00:00, 48.20it/s]


In [104]:
dataset_test =df_test_dummy.drop(["拠点","受付日","品番中グループ","商品品番","商品品目","修理依頼内容","社内メッセージ","得意先メッセージ","一発修理FLG"],axis=1)
dataset_test = dataset_test.drop(["いる","する","なる","てる"],axis=1)

In [105]:
dataset_test 

,受付NO,部品品番,大現象名_その他の不具合,大現象名_便器洗浄関係,大現象名_便座･便蓋関係,大現象名_吐水関係,大現象名_外観関係(ｶﾞﾀﾂｷ含む),大現象名_止水不良(便鉢内漏水),大現象名_漏水,大現象名_異臭関係,大現象名_異音関係,大現象名_脱臭関係,大現象名_電源関係,大現象名_ｼｬﾜｰ･ﾋﾞﾃﾞ関係,大現象名_ﾘﾌﾄｱｯﾌﾟ(ﾁﾙﾄｱｯﾌﾟ)不具合,品番グループ_(G)AWL-71,品番グループ_(G)AWU-506R,品番グループ_(G)AWU-807RAM,品番グループ_(G)DT-3810,品番グループ_(G)DT-3820,品番グループ_(G)DT-3850,品番グループ_(G)DT-3860,品番グループ_(G)DT-3870,品番グループ_(G)DT-3?10HU,品番グループ_(G)DT-4840,...,壁,強,濡れる,開く,出て,よく,フルオート,劣化,量,水音,繋ぎ,つく,周り,スイッチ,上げ,吸い込み,弱く,軸,グラグラ,パイプ,バルブ,調整,掃除,脱臭,左下
0,L2410076,H20-126,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,L2410101,50-2318,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,L2410131,355-1054A,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,L2410173,343-1001D-PSET,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,L2410209,TF-10R-L,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,L2410214,313-1022,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,L2410278,50-1001-2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,L2410278,HH-070,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,L2414799,50-1001-2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,L2415046,372-1013,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [106]:
dataset_test.to_csv('dataset_test_201709.csv')

In [107]:
len(dataset_test)

1191

### ここまで

In [ ]:
print(best)
print(clf.best_score)

チューニング結果を踏まえてもう一度学習

In [ ]:
clf = lgb.LGBMClassifier

clf_isotonic = CalibratedClassifier(clf,cv=stratifiedkfold,method='isotonic')
clf_isotonic.fit(train_x_2,train_y)

#モデルの保存
import pickle

with open('model.pickle',mode='wb') as fp:
    picke.dump(clf,fp)
    
